In [114]:
import pandas as pd; pd.set_option('display.max_columns', 30); pd.set_option('display.max_rows',500)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
exp = 'FR1' # 'catFR1' #'FR1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
exp_df

subs = []
sessions = []
# get list of subs and sessions
for row in exp_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment

        session_raw_log = '/data/eeg/'+sub+'/behavioral/'+exp+'/session_'+str(session)+'/session.json'
        
        raw_log_df = pd.read_json(session_raw_log, lines=True) # just a test...if it exists then will move on
        
        subs.append(sub)
        sessions.append(session)
        
    except:
        dummy = 1
        
print('Unity subs: ')
np.array(subs)
np.array(sessions)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,0,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
1,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,1,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
9,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,0,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
10,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,1,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
17,NaN,protocols/r1/subjects/R1003P/experiments/FR1/s...,protocols/r1/subjects/R1003P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1003P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1003P/localizations/0/m...,NaN,0,R1003P,R1003P,1.0,protocols/r1/subjects/R1003P/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,NaN,protocols/r1/subjects/R1462M/experiments/FR1/s...,protocols/r1/subjects/R1462M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1462M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1462M/localizations/0/m...,NaN,0,R1462M,R1462M,3.4,protocols/r1/subjects/R1462M/experiments/FR1/s...
2151,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,0,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,0,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2152,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,1,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,1,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2166,NaN,protocols/r1/subjects/R1466J/experiments/FR1/s...,protocols/r1/subjects/R1466J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1466J/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1466J/localizations/0/m...,NaN,0,R1466J,R1466J,3.4,protocols/r1/subjects/R1466J/experiments/FR1/s...


Unity subs: 


array(['R1347D', 'R1347D', 'R1347D', 'R1374T', 'R1374T', 'R1375C',
       'R1376D', 'R1377M', 'R1377M', 'R1377M', 'R1377M', 'R1378T',
       'R1378T', 'R1378T', 'R1378T', 'R1379E', 'R1379E', 'R1380D',
       'R1381T', 'R1382T', 'R1383J', 'R1384J', 'R1384J', 'R1384J',
       'R1385E', 'R1385E', 'R1385E', 'R1386T', 'R1386T', 'R1387E',
       'R1387E', 'R1387E', 'R1390M', 'R1391T', 'R1391T', 'R1391T',
       'R1393T', 'R1394E', 'R1395M', 'R1395M', 'R1395M', 'R1395M',
       'R1395M', 'R1395M', 'R1396T', 'R1396T', 'R1397D', 'R1397D',
       'R1398J', 'R1398J', 'R1401J', 'R1402E', 'R1404E', 'R1404E',
       'R1405E', 'R1406M', 'R1406M', 'R1409D', 'R1409D', 'R1412M',
       'R1414E', 'R1415T', 'R1416T', 'R1416T', 'R1420T', 'R1420T',
       'R1421M', 'R1423E', 'R1427T', 'R1427T', 'R1427T', 'R1431J',
       'R1433E', 'R1436J', 'R1438M', 'R1441T', 'R1441T', 'R1441T',
       'R1443D', 'R1443D', 'R1446T', 'R1447M', 'R1448T', 'R1449T',
       'R1449T', 'R1454M', 'R1457T', 'R1457T', 'R1459M', 'R145

array([2, 3, 4, 0, 1, 0, 0, 0, 1, 3, 4, 0, 1, 2, 3, 0, 1, 1, 0, 0, 0, 0,
       1, 2, 1, 2, 3, 0, 1, 0, 1, 2, 1, 0, 1, 2, 0, 0, 0, 1, 2, 3, 4, 5,
       0, 1, 0, 1, 1, 3, 0, 0, 0, 1, 0, 3, 4, 1, 2, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 2, 0, 0, 1, 1, 0, 1, 2, 4, 5, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 6, 0, 0, 0, 0, 0])

In [116]:
stars_df = pd.DataFrame(columns=['subject','session','stars_length','stars_on','stars_off'])

types = ['text display cleared', 'display recall text']

for i,sub in enumerate(subs):
    session = sessions[i]

    session_raw_log = '/data/eeg/'+sub+'/behavioral/'+exp+'/session_'+str(session)+'/session.json'

    raw_log_df = pd.read_json(session_raw_log, lines=True)

    raw_log_df = (pd.DataFrame(raw_log_df['data'].values.tolist())
                    .add_prefix('data.')
                    .join(raw_log_df.drop('data', 1))
                    .query(f"type in {types}")
                    .sort_values('time'))
    mask = raw_log_df["type"] == "display recall text"

    temp_diffs = raw_log_df["time"][mask.shift(periods=1).fillna(False)].reset_index() - raw_log_df["time"][mask].reset_index()
    temp_diffs = temp_diffs.time.values

    stars_df = stars_df.append(pd.DataFrame(data=
                {'subject':np.tile(sub,len(temp_diffs)),
                 'session':np.tile(session,len(temp_diffs)),
                 'stars_length':temp_diffs,
                 'stars_on':raw_log_df["time"][mask].reset_index().time.values,
                 'stars_off':raw_log_df["time"][mask.shift(periods=1).fillna(False)].reset_index().time.values
                }))

In [119]:
stars_df[(stars_df.stars_length>0) & (stars_df.stars_length<990)]
stars_df.stars_length.unique()
stars_df.subject.unique()

,subject,session,stars_length,stars_on,stars_off


array([1017., 1016., 1000., 1001., 1018., 1015.,  999., 1002., 1014.,
       1125.])

array(['R1347D', 'R1374T', 'R1375C', 'R1376D', 'R1377M', 'R1378T',
       'R1379E', 'R1380D', 'R1381T', 'R1382T', 'R1383J', 'R1384J',
       'R1385E', 'R1386T', 'R1387E', 'R1390M', 'R1391T', 'R1393T',
       'R1394E', 'R1395M', 'R1396T', 'R1397D', 'R1398J', 'R1401J',
       'R1402E', 'R1404E', 'R1405E', 'R1406M', 'R1409D', 'R1412M',
       'R1414E', 'R1415T', 'R1416T', 'R1420T', 'R1421M', 'R1423E',
       'R1427T', 'R1431J', 'R1433E', 'R1436J', 'R1438M', 'R1441T',
       'R1443D', 'R1446T', 'R1447M', 'R1448T', 'R1449T', 'R1454M',
       'R1457T', 'R1459M', 'R1460M', 'R1461T', 'R1462M', 'R1466J',
       'R1467M'], dtype=object)

In [120]:
stars_df[stars_df.subject=='R1385E']

,subject,session,stars_length,stars_on,stars_off
0,R1385E,1,1016.0,1.515604e+12,1.515604e+12
1,R1385E,1,1001.0,1.515604e+12,1.515604e+12
2,R1385E,1,1016.0,1.515604e+12,1.515604e+12
3,R1385E,1,1000.0,1.515604e+12,1.515604e+12
4,R1385E,1,1016.0,1.515604e+12,1.515604e+12
5,R1385E,1,1017.0,1.515604e+12,1.515604e+12
6,R1385E,1,1000.0,1.515604e+12,1.515604e+12
7,R1385E,1,1017.0,1.515604e+12,1.515604e+12
8,R1385E,1,1000.0,1.515604e+12,1.515604e+12
9,R1385E,1,1017.0,1.515605e+12,1.515605e+12
